In [1]:
from game_env_copy import GameEnv
import torch
import torch.nn as nn
import tqdm
import numpy as np

### Control Environment

In [2]:
env = GameEnv((20,20), (3,3), (18,18))

In [3]:
def layer_init(layer, bias_const=0.0):
    """Initialize the weights and biases of a layer.

    Args:
        layer (nn.Module): The layer to initialize.
        std (float): Standard deviation for orthogonal initialization.
        bias_const (float): Constant value for bias initialization.

    Returns:
        nn.Module: The initialized layer.
    """
    torch.nn.init.xavier_uniform_(layer.weight)  # Orthogonal initialization
    torch.nn.init.constant_(layer.bias, bias_const)  # Constant bias
    return layer

In [4]:
class Agent(nn.Module):
    def __init__(self, actor_input_size, critic_input_size, actor_output_size, critic_output_size):
        super(Agent, self).__init__()
        hidden_size1, hidden_size2, hidden_size3 = 64, 128, 64
        self.actor = nn.Sequential(
            layer_init(nn.Linear(actor_input_size, hidden_size1)),
            nn.LeakyReLU(),
            layer_init(nn.Linear(hidden_size1, hidden_size2)),
            nn.LeakyReLU(),
            layer_init(nn.Linear(hidden_size2, hidden_size3)),
            nn.LeakyReLU(),
            layer_init(nn.Linear(hidden_size3, actor_output_size))
        )
        self.critic = nn.Sequential(
            layer_init(nn.Linear(critic_input_size, hidden_size1)),
            nn.LeakyReLU(),
            layer_init(nn.Linear(hidden_size1, hidden_size2)),
            nn.LeakyReLU(),
            layer_init(nn.Linear(hidden_size2, hidden_size3)),
            nn.LeakyReLU(),
            layer_init(nn.Linear(hidden_size3, critic_output_size))
        )

    def get_value(self, state):
        return self.critic(state)

    def get_action_probs(self, state):
        return torch.distributions.categorical.Categorical(logits = self.actor(state))

    def get_action(self, probs):
        return probs.sample()

    def get_action_logprob(self, probs, action):
        return probs.log_prob(action)

    def get_entropy(self, probs):
        return probs.entropy()

    def get_action_logprob_entropy(self, state):
        probs = self.get_action_probs(state)
        action = self.get_action(probs)
        logprob = self.get_action_logprob(probs, action)
        entropy = self.get_entropy(probs)
        return action, logprob, entropy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

agent_final = Agent(404, 404, 4, 1).to(device)
agent_adv = Agent(404, 404, 4, 1).to(device)
agent_trunc = Agent(404, 404, 4, 1).to(device)
agent_less = Agent(404, 404, 4, 1).to(device)
agent_op = Agent(404, 404, 4, 1).to(device)

agent_final.load_state_dict(torch.load('less_steps_model/model_final.pth', map_location=device))
agent_adv.load_state_dict(torch.load('less_steps_model/model_adversary.pth', map_location=device))
agent_trunc.load_state_dict(torch.load('less_steps_model/model_trunc.pth', map_location=device))
agent_less.load_state_dict(torch.load('less_steps_model/model_less.pth', map_location=device))
agent_op.load_state_dict(torch.load('model_op.pth', map_location=device))

<All keys matched successfully>

In [5]:
agents = [agent_final, agent_adv, agent_trunc, agent_less, agent_op]

In [6]:
def test(agent, env=GameEnv((20,20), (3,3), (18,18)), num_eps=10):
    rewards = []
    goal_reached = np.zeros(num_eps)

    for episode in tqdm.tqdm(range(num_eps), leave=None):
        total_reward = 0

        state, _ = env.reset()
        for i in range(5000):
            state_tensor = torch.tensor(state,dtype=torch.float32).to(device)

            with torch.no_grad():
                action, _, _ = agent.get_action_logprob_entropy(state_tensor)

            next_state, reward, done, _, infos = env.step(action.cpu().item())
            total_reward += reward

            state = torch.tensor(next_state, dtype=torch.float32, device=device)
            
    #         print(state[-4:])
            
            if done:
                goal_reached[episode] = 1
                break

        rewards.append(total_reward)

    env.close()

    print(f"\nMean Rewards: {np.mean(rewards)}{', never reach goal' if not goal_reached.any() else ''}\n")
    return np.mean(rewards), goal_reached

In [7]:
def test_agents(agents, *args):
    rewards = []
    goal_reached = []
    for agent in agents:
        reward, goal_reach = test(agent, *args)
        rewards.append(reward)
        goal_reached.append(goal_reach)
    return rewards, goal_reached

In [ ]:
results3 = [test_agents(agents, GameEnv((20,20), (np.random.randint(1, 7),np.random.randint(1,7)), (18, 18)), 1) for i in range(10) ]
results4 = [test_agents(agents, GameEnv((20,20), (np.random.randint(1, 21),np.random.randint(1,21)), (np.random.randint(1, 21),np.random.randint(1,21))), 1) for i in range(10) ]
results2= [test_agents(agents, GameEnv((20,20), (3,3), (np.random.randint(12, 21),np.random.randint(12,21))), 1) for i in range(10)]
results1= test_agents(agents, GameEnv((20,20), (3,3), (18, 18)), 10)

In [8]:
results1 = test_agents(agents, GameEnv((20,20), (3,3), (18, 18)), 10) # training env
results2 = test_agents(agents, GameEnv((20, 20), (18, 18), (3, 3)), 10) # swapped start goal
results3 = [test_agents(agents, GameEnv((20,20), (np.random.randint(1, 7),np.random.randint(1,7)), (np.random.randint(12, 21),np.random.randint(12,21))), 1) for i in range(10) ] # up right
results4 = [test_agents(agents, GameEnv((20,20), (np.random.randint(12, 21),np.random.randint(12,21)), (np.random.randint(1, 7),np.random.randint(1,7))), 1) for i in range(10) ] # down left
results5 = [test_agents(agents, GameEnv((20,20), (np.random.randint(1, 7),np.random.randint(12,21)), (np.random.randint(12, 21),np.random.randint(1,7))), 1) for i in range(10) ] # down right
results6 = [test_agents(agents, GameEnv((20,20), (np.random.randint(12, 21),np.random.randint(1,7)), (np.random.randint(1, 7),np.random.randint(12,21))), 1) for i in range(10) ] # up left
results7 = [test_agents(agents, GameEnv((20,20), (np.random.randint(1, 21),np.random.randint(1,21)), (np.random.randint(1, 21),np.random.randint(1,21))), 1) for i in range(10) ] # random

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\Vincent\AppData\Local\Temp\ipykernel_35204\1817032982.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state_tensor = torch.tensor(state,dtype=torch.float32).to(device)
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:36<00:00,  9.64s/it]



Mean Rewards: -44399.3, never reach goal



100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:40<00:00, 10.05s/it]



Mean Rewards: -116974.4, never reach goal



100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:37<00:00,  9.76s/it]


...


Mean Rewards: -31457.0, never reach goal



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.01s/it]



Mean Rewards: -84801.0, never reach goal



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.68s/it]


Mean Rewards: -10390.0



In [9]:
def get_reward_goal(result):
    reward = []
    goal = []
    for res in result:
        reward.append(res[0])
        goal.append(res[1])
    return np.mean(np.array(reward), axis=0), np.hstack(np.array(goal))

In [10]:
tuple(map(np.array, results1))

(array([ -44399.3, -116974.4,  -85074. ,  -95157.1,    1563.6]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]))

In [11]:
tuple(map(np.array, results2))

(array([-26840.1,   1482.3, -24300.3, -95034.4,   1488.1]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]))

In [12]:
get_reward_goal(results3)

(array([ -37823.2, -101224.8,  -79829. ,  -90594.3,  -17883.1]),
 array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0., 1., 0.]]))

In [13]:
get_reward_goal(results4)

(array([-18916.8,  -1366.2, -21977.5, -94028.8,  -6869.2]),
 array([[0., 1., 0., 0., 0., 0., 0., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 1.]]))

In [14]:
get_reward_goal(results5)

(array([-10726.7, -70800.3, -11909.9, -24825.3, -79855.6]),
 array([[0., 1., 0., 1., 1., 1., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))

In [15]:
get_reward_goal(results6)

(array([-129721. ,  -70053.4, -123216.1, -156453. ,  -69474.2]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))

In [16]:
get_reward_goal(results7)

(array([-42540.2, -71122.8, -47409.6, -80757. , -31877.2]),
 array([[1., 0., 0., 0., 0., 0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 0., 1., 1.]]))

In [23]:
agent_new = Agent(404, 404, 4, 1).to(device)
agent_new_less = Agent(404, 404, 4, 1).to(device)

agent_new.load_state_dict(torch.load('new_model/model_final.pth', map_location=device))
agent_new.load_state_dict(torch.load('new_model/model_final_less_step.pth', map_location=device))

<All keys matched successfully>

In [24]:
agents = [agent_new,  agent_new_less]

In [18]:
new_result1 = test(agent_new, env = GameEnv((20,20), (3,3), (18, 18)))
new_result2 = test(agent_new, env = GameEnv((20,20), (3,3), (np.random.randint(12, 21),np.random.randint(12,21))))
new_result3 = test(agent_new, env = GameEnv((20,20), (np.random.randint(1, 7),np.random.randint(1,7)), (18, 18)))
new_result4 = test(agent_new, GameEnv((20,20), (np.random.randint(1, 21),np.random.randint(1,21)), (np.random.randint(1, 21),np.random.randint(1,21))))

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\Vincent\AppData\Local\Temp\ipykernel_80796\1894394970.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state_tensor = torch.tensor(state,dtype=torch.float32).to(device)
100%|██████████| 10/10 [01:56<00:00, 11.65s/it]



Mean Rewards: -96888.4, never reach goal



100%|██████████| 10/10 [01:58<00:00, 11.86s/it]



Mean Rewards: -55771.8, never reach goal



100%|██████████| 10/10 [01:55<00:00, 11.52s/it]



Mean Rewards: -98431.5, never reach goal



100%|██████████| 10/10 [02:04<00:00, 12.47s/it]


Mean Rewards: -98838.8, never reach goal



In [25]:
new_results3 = [test_agents(agents, GameEnv((20,20), (np.random.randint(1, 7),np.random.randint(1,7)), (18, 18)), 1) for i in range(10) ]
new_results4 = [test_agents(agents, GameEnv((20,20), (np.random.randint(1, 21),np.random.randint(1,21)), (np.random.randint(1, 21),np.random.randint(1,21))), 1) for i in range(10) ]
new_results2= [test_agents(agents, GameEnv((20,20), (3,3), (np.random.randint(12, 21),np.random.randint(12,21))), 1) for i in range(10)]
new_results1= test_agents(agents, GameEnv((20,20), (3,3), (18, 18)), 10)

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Vincent\AppData\Local\Temp\ipykernel_80796\1894394970.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state_tensor = torch.tensor(state,dtype=torch.float32).to(device)
100%|██████████| 1/1 [00:13<00:00, 13.13s/it]



Mean Rewards: -19376.0, never reach goal



100%|██████████| 1/1 [00:01<00:00,  1.02s/it]



Mean Rewards: -3154.0



100%|██████████| 1/1 [00:13<00:00, 13.39s/it]


...


Mean Rewards: -587.0



100%|██████████| 10/10 [01:59<00:00, 11.91s/it]



Mean Rewards: -26161.2, never reach goal



100%|██████████| 10/10 [00:05<00:00,  1.98it/s]


Mean Rewards: -1458.5



In [27]:
new_results1

([-26161.2, -1458.5],
 [array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])])

In [30]:
get_reward_goal(new_results2)

(array([-24533.5, -22339.3]),
 array([[0., 0., 0., 0., 1., 1., 0., 0., 1., 0.],
        [0., 1., 0., 1., 0., 1., 1., 1., 1., 1.]]))

In [31]:
get_reward_goal(new_results3)

(array([-20688.9,  -2887.2]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]))

In [ ]:
get_reward_goal(new_results4)

(array([ -25492.6, -110826.1]),
 array([[1., 1., 0., 1., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]))